In [1]:
import torch
from torch.utils.data import DataLoader
from torchvision import transforms
import numpy as np
from custom_dataset import Binary_Dataset
from matplotlib import pyplot as plt
from PIL import Image
import os
from inception import inception_v3
from Trainer import train_loop
from torch.optim import Adam
%matplotlib inline
torch.cuda.empty_cache()

# Define Dataloaders for each class


In [2]:
#define relevant paths
labels_dir="Data\list_attr_celeba.csv"
image_dir="Data\img_align_celeba\img_align_celeba"

############################################define batch size for all data loaders here
bathsize=32

##############################################define num workers for data laoders here
workers=4

#define composed transforms
composed = transforms.Compose([
    transforms.ToPILImage(),
    transforms.Resize([299,299]),
    transforms.ToTensor(),
    transforms.Normalize(mean=[0.485, 0.456, 0.406], std=[0.229, 0.224, 0.225])
    ])

labels=["No_Beard","Male","Bald","Bangs","Smiling"]
label_number=1
#Choose which label 
ds=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Train")
dataset_loader=DataLoader(ds,batch_size=bathsize, shuffle=True, num_workers=workers)

ds_val=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Val")
val_loader=DataLoader(ds_val,batch_size=bathsize, shuffle=True, num_workers=workers)

"""
# male
ds1=Binary_Dataset(labels_dir, labels[1],image_dir ,transform=composed)
dataset_loader=DataLoader(ds1,batch_size=bathsize, shuffle=True, num_workers=workers)

# bald
ds2=Binary_Dataset(labels_dir, labels[2],image_dir ,transform=composed)
dataset_loader=DataLoader(ds2,batch_size=bathsize, shuffle=True, num_workers=workers)

# bangs
ds3=Binary_Dataset(labels_dir, labels[3],image_dir ,transform=composed)
dataset_loader=DataLoader(ds3,batch_size=bathsize, shuffle=True, num_workers=workers)

# smiling
ds4=Binary_Dataset(labels_dir, labels[4],image_dir ,transform=composed)
dataset_loader=DataLoader(ds4,batch_size=bathsize, shuffle=True, num_workers=workers)
"""

"""
for idx, (data, image) in enumerate(ds):
    print(idx)
"""


data=next(iter(dataset_loader))
sample, target = data
#print(sample)
print(target)


tensor([1, 1, 1, 1, 1, 0, 1, 0, 0, 0, 1, 0, 0, 1, 1, 0, 1, 1, 1, 1, 1, 0, 0, 0,
        0, 0, 0, 1, 0, 0, 1, 1])


In [3]:
"""
#wont work anymore because pil
idx=0
plt.imshow(sample[idx])
print(labels[label_number])
print(target[idx])
"""

'\n#wont work anymore because pil\nidx=0\nplt.imshow(sample[idx])\nprint(labels[label_number])\nprint(target[idx])\n'

# Define CNN's

In [4]:
device = torch.device("cuda:0" if torch.cuda.is_available() else "cpu")
model=inception_v3(pretrained=False).to(device)

# Training Loop

In [5]:
learning_rate=0.0001
optimizer = Adam(params=model.parameters(), lr=learning_rate)
train_loop(model=model,dataloader=dataset_loader,epochs=1,optimizer=optimizer,save_name="Male_12_8")

tensor(125.0629, device='cuda:0', grad_fn=<AddBackward0>)
tensor(97.4640, device='cuda:0', grad_fn=<AddBackward0>)
tensor(94.4030, device='cuda:0', grad_fn=<AddBackward0>)
tensor(87.2239, device='cuda:0', grad_fn=<AddBackward0>)
tensor(80.8113, device='cuda:0', grad_fn=<AddBackward0>)
tensor(77.3148, device='cuda:0', grad_fn=<AddBackward0>)
tensor(73.1976, device='cuda:0', grad_fn=<AddBackward0>)
tensor(71.5639, device='cuda:0', grad_fn=<AddBackward0>)
tensor(69.0564, device='cuda:0', grad_fn=<AddBackward0>)
tensor(63.8872, device='cuda:0', grad_fn=<AddBackward0>)
tensor(57.1163, device='cuda:0', grad_fn=<AddBackward0>)
tensor(55.0326, device='cuda:0', grad_fn=<AddBackward0>)
tensor(54.0632, device='cuda:0', grad_fn=<AddBackward0>)
tensor(54.7323, device='cuda:0', grad_fn=<AddBackward0>)
tensor(55.7831, device='cuda:0', grad_fn=<AddBackward0>)
tensor(53.2856, device='cuda:0', grad_fn=<AddBackward0>)
tensor(46.3569, device='cuda:0', grad_fn=<AddBackward0>)
tensor(48.4417, device='cuda:0

KeyboardInterrupt: 

In [6]:
#check validation accuracy
ds_val=Binary_Dataset(labels_dir, labels[label_number],image_dir ,transform=composed, partition="Val")
val_loader=DataLoader(ds_val,batch_size=bathsize, shuffle=True, num_workers=workers)
device = torch.device("cpu")
val_data=iter(val_loader)
model=model.to(device)
correct=0
tested_samples=0
limit=2000
for data in val_data:
    sample, target = data
    
    #send to device
    sample=sample.to(device)
    target=target.to(device)

    pred=torch.argmax(model(sample),dim=1,keepdim=True)

    correct+=torch.eq(pred,target.view_as(pred)).sum()
    tested_samples+=pred.size()[0]
    if tested_samples>limit:
        break

    print(correct)

#print precision
print(correct.item()/tested_samples)

tensor(30)
tensor(59)
tensor(87)
tensor(117)
tensor(146)
tensor(175)
tensor(202)
tensor(230)
tensor(260)
tensor(287)
tensor(315)
tensor(344)
tensor(373)
tensor(402)
tensor(432)
tensor(459)
tensor(489)
tensor(517)
tensor(543)
tensor(572)
tensor(602)
tensor(629)
tensor(658)
tensor(687)
tensor(716)
tensor(746)
tensor(776)
tensor(806)
tensor(837)
tensor(864)
tensor(896)
tensor(925)
tensor(957)
tensor(987)
tensor(1016)
tensor(1047)
tensor(1076)
tensor(1105)
tensor(1134)
tensor(1165)
tensor(1195)
tensor(1224)
tensor(1252)
tensor(1280)
tensor(1309)
tensor(1338)
tensor(1367)
tensor(1392)
tensor(1421)
tensor(1448)
tensor(1475)
tensor(1504)
tensor(1532)
tensor(1564)
tensor(1594)
tensor(1626)
tensor(1655)
tensor(1684)
tensor(1712)
tensor(1742)
tensor(1773)
tensor(1801)
0.908234126984127


0.6796875
